# Check GPU info

In [1]:
!nvidia-smi

Thu Aug 24 16:25:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080        On  | 00000000:01:00.0 Off |                  N/A |
|  0%   38C    P2              35W / 198W |     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Install and import dependencies

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install jsons
!pip install langchain
!pip install textwrap3

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number

In [4]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from langchain.document_loaders import PyPDFLoader
from transformers import pipeline
import json
import textwrap

/home/voyagerke/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Ingest pdf file

In [8]:
pdf_file_path = "./finance.pdf"
# pdf_file_path = "./attention is all you need.pdf"

In [9]:
pdf_loader = PyPDFLoader(pdf_file_path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splitted_docs = text_splitter.split_documents(pdf_loader.load())

In [43]:
print(splitted_docs[0])

page_content='中信证券研报表示，选取了14家具有代表性的海外CRO/CDMO企业以及生命科学上\n游服务商，对其中报业绩进行总结并作为对国内企业的参考。中信证券发现，全球临床\n阶段的新药研发依然活跃，早期研发的需求虽然暂时疲软，但随着海外投融资的回暖，\n也有望回归快速增长常态。生命科学上游服务商的收入，则受到了下游企业去库存周期\n的影响，但相关公司也普遍认为四季度将出现新增需求的拐点。综上，中信证券认为一\n体化服务于全球中后期临床研发阶段/商业化阶段外包需求，以及高校/科研院所客户收\n入占比高的企业业绩相对稳定，且海外收入占比较高的企业将有望率先受益于投融资回\n暖带来的需求增加。\n全文如下' metadata={'source': './finance.pdf', 'page': 0}


# Build and store embeddings

In [11]:
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install Chromadb

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number

## Build Sentence Embeddings for Chinese

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

'''
https://huggingface.co/shibing624/text2vec-base-chinese/tree/main
'''
chinese_embedding_name = "/mnt/h/text2vec-base-chinese"
embeddings = HuggingFaceEmbeddings(
    model_name=chinese_embedding_name,
    model_kwargs={"device": "cuda"},
)

In [29]:
collection_name = 'llama2_demo'
db = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory='./'
)


In [31]:
db.add_documents(splitted_docs)

['946cabe6-4258-11ee-a0ca-817489a76db9',
 '946cabe7-4258-11ee-a0ca-817489a76db9',
 '946cabe8-4258-11ee-a0ca-817489a76db9',
 '946cabe9-4258-11ee-a0ca-817489a76db9',
 '946cabea-4258-11ee-a0ca-817489a76db9',
 '946cabeb-4258-11ee-a0ca-817489a76db9',
 '946cabec-4258-11ee-a0ca-817489a76db9',
 '946cabed-4258-11ee-a0ca-817489a76db9',
 '946cabee-4258-11ee-a0ca-817489a76db9',
 '946cabef-4258-11ee-a0ca-817489a76db9',
 '946cabf0-4258-11ee-a0ca-817489a76db9',
 '946cabf1-4258-11ee-a0ca-817489a76db9',
 '946cabf2-4258-11ee-a0ca-817489a76db9']

## Test similarity search in Chroma

In [32]:
test_query = '医疗健康投资的风险因素有哪些？'
# test_query = 'what is dominant sequence transduction models?'
search_docs = db.similarity_search(test_query)
print(search_docs)

[Document(page_content='医疗健康｜海外CXO生命科学上游公司中报盘点：投融资回暖趋势已现，需求拐\n点在望\n我们选取了14家具有代表性的海外CRO/CDMO企业以及生命科学上游服务商，\n对其中报业绩进行总结并作为对国内企业的参考。我们发现，全球临床阶段的新药研发\n依然活跃，早期研发的需求虽然暂时疲软，但随着海外投融资的回暖，也有望回归快速\n增长常态。生命科学上游服务商的收入，则受到了下游企业去库存周期的影响，但相关\n公司也普遍认为23Q4将出现新增需求的拐点。综上，我们认为一体化服务于全球中后\n期临床研发阶段/商业化阶段外包需求，以及高校/科研院所客户收入占比高的企业业绩', metadata={'page': 0, 'source': './finance.pdf'}), Document(page_content='增长常态。生命科学上游服务商的收入，则受到了下游企业去库存周期的影响，但企业\n普遍认为23Q4将出现新增需求的拐点。综上，我们认为一体化服务于全球中后期临床\n研发阶段/商业化阶段外包需求，以及高校/科研院所/大药企客户收入占比高的企业业\n绩相对稳定；且海外收入占比较高的企业将有望率先受益于投融资回暖带来的需求增加。', metadata={'page': 4, 'source': './finance.pdf'}), Document(page_content='了项目投资计划。来自资金相对更为充沛的高校、科研院所、大药企的需求则更为稳定。\n我们认为投融资的改善有望促进下游企业的资本开支恢复常态。\n3）中国区需求收入有一定波动。2023年H1，海外企业均表示中国区需求出现了\n下降，这一部分是因为国内同样正在经历投融资下滑+去库存的周期，一部分则是因为\n中国本土供应商的崛起：Sartorius认为中国本土供应商的市场份额已显著上升。\n▍风险因素：\n生物医药投融资恢复不及预期；去库存周期长于预期；宏观经济恢复不及预期；研\n发投入不及预期；研发管线推进不及预期。\n▍投资策略：\n我们选取了14家具有代表性的海外CRO/CDMO企业以及生命科学上游服务商，', metadata={'page': 3, 'source': './finance.pdf'}), Document(page_content

In [16]:
# db.delete_collection()

# Build model and transformer pipeline

In [17]:
!pip install accelerate
!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Exp

In [18]:
'''
https://huggingface.co/LinkSoul/Chinese-Llama-2-7b-4bit
'''
model_path = '/mnt/h/Chinese-Llama-2-7b-4bit'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:36<00:00, 48.41s/it]


In [19]:
generation_config = GenerationConfig.from_pretrained(model_path)
pipe = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## The function for building prompt

In [20]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [21]:
# Test function get_prompt()

instruction = "What is the temperature in Melbourne?"
get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is the temperature in Melbourne?[/INST]"

## Build HuggingFacePipeline

In [22]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [23]:
def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text +'\n\n')

In [24]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [25]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer,\
just say that you don't know, don't try to make up an answer. Must use Chinese to answer the question.

{context}

{history}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key='question', memory_key='history')

In [34]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)
# retriever_from_llm = db.as_retriever()

## Add context compression

Max token size of Llama2 is 2048, should compress each document.

In [35]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)

In [37]:
import logging
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.DEBUG)

In [38]:
retri_docs = compression_retriever.get_relevant_documents('医疗健康投资的风险因素有哪些？')
print(retri_docs)

/home/voyagerke/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[Document(page_content='"全球临干阶段的新药研发依然活跃"', metadata={'page': 0, 'source': './finance.pdf'}), Document(page_content='"生命科学上游服务商的收入，则受到了下游企业去库存周期的影响"', metadata={'page': 0, 'source': './finance.pdf'}), Document(page_content='"生命科学上游服务商的收入，则受到了下游企业去库存周期的影响"', metadata={'page': 4, 'source': './finance.pdf'}), Document(page_content='"全球生物医药领域的投融资已出现回暖"', metadata={'page': 2, 'source': './finance.pdf'}), Document(page_content='"生物医药投融资恢复不及预期" and "去库存周期长于预期".', metadata={'page': 3, 'source': './finance.pdf'}), Document(page_content='Repligen, Waters, MerckKGaA, Sartorius are representatives of overseas life science upstream suppliers.', metadata={'page': 2, 'source': './finance.pdf'}), Document(page_content='IQVIA, Medpace, CRO, Q2, RFP, business growth, new contracts, revenue increase, client engagement,\noverall performance, industry outlook.', metadata={'page': 1, 'source': './finance.pdf'})]


In [41]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = compression_retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt, "memory": memory}
)

# Q&A

In [42]:
query = input("\nEnter a query: ")
res = qa(query)
print(res)


Enter a query:  医疗健康投资的风险因素有哪些？


{'query': '医疗健康投资的风险因素有哪些？', 'result': ' 在医疗健康行业中进行投资时，可能会面临以下一些主要风险因素：1）政府法规和监管变化；2）技术创新与市场需求的失衡；3）公司治理问题；4）经济波动对消费者支付力度的影响；5）人才流失等。此外，还应注意其他特定行业或项目的风险，如基因工程、生物制品安全性等方面的风险。 ', 'source_documents': [Document(page_content='"全球临干阶段的新药研发依然活跃"', metadata={'page': 0, 'source': './finance.pdf'}), Document(page_content='"生命科学上游服务商的收入，则受到了下游企业去库存周期的影响"', metadata={'page': 0, 'source': './finance.pdf'}), Document(page_content='"生命科学上游服务商的收入，则受到了下游企业去库存周期的影响"', metadata={'page': 4, 'source': './finance.pdf'}), Document(page_content='"全球生物医药领域的投融资已出现回暖"', metadata={'page': 2, 'source': './finance.pdf'}), Document(page_content='"生物医药投融资恢复不及预期" and "去库存周期长于预期".', metadata={'page': 3, 'source': './finance.pdf'}), Document(page_content='Repligen, Waters, MerckKGaA, Sartorius are representatives of overseas life science upstream suppliers.', metadata={'page': 2, 'source': './finance.pdf'}), Document(page_content='IQVIA, Medpace, CRO, Q2, RFP, business growth, new contracts, revenue increase, client eng